I took a handful of audio files that I had, matched them to spotify (to get the spotify estimated BPM, and then ran them through the same algorithms from the "BPM analysis" notebook

Perhaps Jazz and Prog Rock aren't the best candidates for a stable BPM estimation, and results show that there is still some variation. We likely need a more broad dataset of test tracks, and some way to compare them with human consensus too.

In [1]:
import os
import json
import statistics
import concurrent.futures

import tabulate
from IPython.core.display import HTML

import vamp
import librosa

import essentia.standard as es

from tempocnn.classifier import TempoClassifier, MeterClassifier
from tempocnn.feature import read_features

from madmom.features import (RNNBeatProcessor, TempoEstimationProcessor)

[   INFO   ] MusicExtractorSVM: no classifier models were configured by default


In [2]:
def compute_bpm_vamp(file):
    data, rate = librosa.load(file)
    beats = vamp.collect(data, rate, "qm-vamp-plugins:qm-tempotracker")
    vamp_beats = [float(b["label"].replace(" bpm", "")) for b in beats["list"] if b["label"]]
    vamp_bpm = round(statistics.mean(vamp_beats), 2)
    return {"vamp": vamp_bpm}

def compute_bpm_essentia(file):
    audio = es.MonoLoader(filename=file)()

    # Slow
    rhythm_extractor = es.RhythmExtractor2013(method="multifeature")
    multifeature_bpm, beats, beats_confidence, _, beats_intervals = rhythm_extractor(audio)

    # faster
    rhythm_extractor = es.RhythmExtractor2013(method="degara")
    degara_bpm, beats, beats_confidence, degara_bpms, beats_intervals = rhythm_extractor(audio)

    # PercivalBpmEstimator
    percival_bpm = es.PercivalBpmEstimator()(audio)

    return {"ess_mutifeature": multifeature_bpm, "ess_degara": degara_bpm, "ess_percival": percival_bpm}

def compute_bpm_tempocnn(file):
    model = 'fcn' # default
    classifier = TempoClassifier(model)

    features = read_features(file)
    tempo = classifier.estimate_tempo(features, interpolate=False)
    return {"tempocnn": tempo}

def compute_bpm_madmom(file):
    proc = RNNBeatProcessor()
    tempo = TempoEstimationProcessor(method='comb', min_bpm=40., fps=100,
                                               max_bpm=250., act_smooth=0.14,
                                               hist_smooth=9, hist_buffer=10.,
                                               alpha=0.79, visualize=False)
    tempo.visualize = False
    beats = proc(file)
    tempo = tempo.process_online(beats)
    return {"madmom": round(tempo.tolist()[0][0])}

In [3]:
j = json.load(open("metadata_and_spotify.json"))

In [9]:
def make_table(source, results):
    directory = [i for i in j if i['root'] == source][0]
    header = ["file", "vamp", "ess_mutifeature", "ess_degara", "ess_percival", "tempocnn", "madmom", "spotify", "spotify name", "spotify link"]
    data = [header]
    for file, spot in zip(directory["files"], directory["spotify"]["tracks"]):
        res = results[file]
        row = [file]
        for alg in ["vamp", "ess_mutifeature", "ess_degara", "ess_percival", "tempocnn", "madmom"]:
            row.append(int(round(res[alg], 0)))
        feat = spot["features"][0]
        link = f'''<a href="https://open.spotify.com/track/{feat['id']}">{feat['id']}</a>'''
        row.extend([feat["tempo"], spot['name'], link])
        data.append(row)
    return data

In [6]:
all_results.keys()

dict_keys(['music/Miles Davis - 1959 - Kind of Blue', 'music/Pink Floyd - 1975 - Wish You Were Here', 'music/Pink Floyd - 1973 - The Dark Side of the Moon', 'music/Pink Floyd - 1979 - The Wall', 'music/Queen - 1981 - Greatest Hits'])

In [10]:
k = 'music/Pink Floyd - 1975 - Wish You Were Here'
r = all_results[k]
data = make_table(k, r)
table = tabulate.tabulate(data, tablefmt='unsafehtml')
table

file,vamp,ess_mutifeature,ess_degara,ess_percival,tempocnn,madmom,spotify,spotify name,spotify link
01 - Pink Floyd - Shine On You Crazy Diamond (Part One).flac,142,136,136,69,46,136,137.937,Shine On You Crazy Diamond (Pts. 1-5),6pnwfWyaWjQiHCKTiZLItr
02 - Pink Floyd - Welcome to the Machine.flac,136,133,133,67,133,133,133.271,Welcome to the Machine,5VWC7v2dC2K0SIIjT9WTLN
03 - Pink Floyd - Have a Cigar.flac,120,123,122,121,122,120,120.155,Have a Cigar,3CmHvyZQQAGkKkTjTBFWN6
04 - Pink Floyd - Wish You Were Here.flac,121,121,120,122,61,120,122.861,Wish You Were Here,6mFkJmJqdDVQ1REhVfGgd1
05 - Pink Floyd - Shine On You Crazy Diamond (Part Two).flac,138,165,165,88,42,133,135.87,Shine On You Crazy Diamond (Pts. 6-9),21j1PsCiTaO8ZW88UZrh3A


In [11]:
k = 'music/Pink Floyd - 1979 - The Wall'
r = all_results[k]
data = make_table(k, r)
table = tabulate.tabulate(data, tablefmt='unsafehtml')
table

file,vamp,ess_mutifeature,ess_degara,ess_percival,tempocnn,madmom,spotify,spotify name,spotify link
01-01 - Pink Floyd - In the Flesh_.flac,144,152,152,77,51,154,152.1,In the Flesh?,24YmWQgunJGAU8El3ndkyn
01-02 - Pink Floyd - The Thin Ice.flac,115,132,132,92,132,66,110.272,The Thin Ice,62zuXt6X5B6YQOBNFV2eXG
"01-03 - Pink Floyd - Another Brick in the Wall, Part 1.flac",103,99,99,99,100,98,99.524,"Another Brick in the Wall, Pt. 1",7K6xMPtAjTuLPNlJMLf5bS
01-04 - Pink Floyd - The Happiest Days of Our Lives.flac,104,103,103,104,104,103,103.994,The Happiest Days of Our Lives,2O3l4X1yTua8oMMCtazkyo
"01-05 - Pink Floyd - Another Brick in the Wall, Part 2.flac",103,103,103,104,104,103,104.114,"Another Brick in the Wall, Pt. 2",4gMgiXfqyzZLMhsksGmbQV
01-06 - Pink Floyd - Mother.flac,139,135,135,135,67,67,135.22,Mother,1GEOSS415bZVHNuXWlCT6b
01-07 - Pink Floyd - Goodbye Blue Sky.flac,117,109,109,101,108,109,109.154,Goodbye Blue Sky,0ESdtt9cjGZUkUbaubSrv2
01-08 - Pink Floyd - Empty Spaces.flac,96,95,95,97,95,95,95.942,Empty Spaces,5yMyLZu4wPvWcbLFiRhYVM
01-09 - Pink Floyd - Young Lust.flac,95,95,95,95,95,95,95.488,Young Lust,3xGJuHvSxFJxxYlHj5BIoT
01-10 - Pink Floyd - One of My Turns.flac,158,172,172,89,87,44,175.868,One of My Turns,66YHxyGAUGjMBDdrDlWIMD


In [12]:
k = 'music/Queen - 1981 - Greatest Hits'
r = all_results[k]
data = make_table(k, r)
table = tabulate.tabulate(data, tablefmt='unsafehtml')
table

file,vamp,ess_mutifeature,ess_degara,ess_percival,tempocnn,madmom,spotify,spotify name,spotify link
01 - Queen - Bohemian Rhapsody.mp3,144,140,140,71,71,71,143.353,Bohemian Rhapsody - Remastered 2011,6ljkRMigoNtu0x1mlTEsKc
02 - Queen - Another One Bites the Dust.mp3,110,110,110,110,110,109,109.963,Another One Bites The Dust - Remastered 2011,6hfNDGNTJBR029RmV63IoO
03 - Queen - Killer Queen.mp3,117,117,118,117,116,118,117.12,Killer Queen - Remastered 2011,6Oj0XnWrDEl3KrwZuMQqVj
04 - Queen - Fat Bottomed Girls.mp3,90,89,89,89,87,88,88.176,Fat Bottomed Girls - Single Version / Remastered 2011,52ZQTzXbbWjS4kjOcV3z5b
05 - Queen - Bicycle Race.mp3,145,164,165,84,83,83,84.364,Bicycle Race - Remastered 2011,6I55r9WyH1wV1whBRacLFa
06 - Queen - You’re My Best Friend.mp3,120,118,120,120,118,120,118.741,You're My Best Friend - Remastered 2011,6CVzXxIHDIDdyzlgfEWSZr
07 - Queen - Don’t Stop Me Now.mp3,163,156,156,78,157,52,156.395,Don't Stop Me Now - Remastered 2011,064C5ivM2FUsY0ghkyt4YK
08 - Queen - Save Me.mp3,86,82,82,82,82,81,81.63,Save Me - Remastered 2011,5V890judRbpVT6X5AEYZc8
09 - Queen - Crazy Little Thing Called Love.mp3,153,153,153,153,76,154,76.981,Crazy Little Thing Called Love - Remastered 2011,0q8IUBbw0iedjCbzs7vT6U
10 - Queen - Somebody to Love.mp3,126,73,110,73,73,109,112.568,Somebody To Love - Remastered 2011,4RJdwSqHapVcW5DaRtTkv0


In [13]:
k = 'music/Miles Davis - 1959 - Kind of Blue'
r = all_results[k]
data = make_table(k, r)
table = tabulate.tabulate(data, tablefmt='unsafehtml')
table

file,vamp,ess_mutifeature,ess_degara,ess_percival,tempocnn,madmom,spotify,spotify name,spotify link
01 - Miles Davis - So What.flac,138,137,137,137,135,68,136.526,"So What (feat. John Coltrane, Cannonball Adderley & Bill Evans)",7q3kkfAVpmcZ8g6JUThi3o
02 - Miles Davis - Freddie Freeloader.flac,130,129,129,128,130,128,128.453,"Freddie Freeloader (feat. John Coltrane, Cannonball Adderley, Wynton Kelly & Paul Chambers)",3NvYPUNu6nwQgN31UnoDbn
03 - Miles Davis - Blue in Green.flac,136,112,113,97,57,57,170.272,Blue in Green (feat. John Coltrane & Bill Evans),6bP4GyrKNbcKPMDqWJqpxI
04 - Miles Davis - All Blues.flac,139,139,139,140,139,140,140.033,"All Blues (feat. John Coltrane, Cannonball Adderley & Bill Evans)",0TUa7gBg7LJH6B8dkNVioU
05 - Miles Davis - Flamenco Sketches.flac,145,117,117,118,116,59,175.137,"Flamenco Sketches (feat. John Coltrane, Cannonball Adderley & Bill Evans)",3R3fcrKaeBr3U6s9R2belr
